# Adversarial Attacks

In [ ]:
%load_ext autoreload
%autoreload 2

import os 

PROJECT_DPATH = os.path.abspath(os.path.join(os.curdir, os.pardir))
DATA_DPATH = os.path.join(PROJECT_DPATH, "data")

import torch
import torchvision
from torchvision.transforms import transforms 
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from PIL import Image
from datetime import date
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt

from mnist_recognition.models import MlpModel
from mnist_recognition.inference import Inference
from mnist_recognition.evaluation import Evaluator
from mnist_recognition.transforms import Invertor
from mnist_recognition.fgsm_attack import fgsm_attack

## Загрузка тестовой выборки

In [ ]:
# transform = transforms.Compose([Invertor(),transforms.Compose([transforms.ToTensor()])])
transform = transforms.Compose(
    [
        Invertor(),
        transforms.ToTensor()
    ]
)
# загружаем тестовую выборку
test_data = torchvision.datasets.MNIST(
    DATA_DPATH, train=False, transform=transform, download=True
)

test_dataloader=torch.utils.data.DataLoader(
    dataset=test_data, 
    batch_size=1,
    shuffle=False
)

## Загрузка обученной модели

In [ ]:
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_dpath = os.path.join(PROJECT_DPATH, "checkpoints")
model_fpath = os.path.join(checkpoint_dpath, "best_valid_with_augmentations.pth")

infer = Inference.from_file(model_fpath, device= DEVICE)

model = MlpModel()
model = model.to(DEVICE)
model.load_state_dict(torch.load(model_fpath)["model_state"])

In [ ]:
def data_fgsm_attack(model, device, test_loader, epsilon ):
    output_data=[]
    origin_data=[]
    # Loop over all examples in test set
    for data, target in tqdm(test_loader):
        # Send the data and label to the device
        data, target = data.to(device), target.to(device)
        
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data.reshape(-1, 28 * 28))
        init_pred = output.max(1, keepdim=True)[1]# get the index of the max log-probability
        
        # If the initial prediction is wrong, dont bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        # Calculate the loss
        loss = F.nll_loss(output, target)

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect datagrad
        data_grad = data.grad.data
        
        # Call FGSM Attack
        perturbed_data = fgsm_attack(data.reshape(-1, 28 * 28), epsilon, data_grad.reshape(-1, 28 * 28))

        # Transform tensor to type PIL.Image.Image 
        perturbed_data = np.reshape(perturbed_data.cpu().detach().numpy(),(28,28))
        perturbed_data = Image.fromarray(perturbed_data)

        output_data.append([perturbed_data,target.item()])

        # Collect origin data examples for visualisation 
        original_img = np.reshape(data.cpu().detach().numpy(),(28,28))
        original_img = Image.fromarray(original_img)
        origin_data.append([original_img,target.item()])

    return output_data, origin_data


Рассмотрим влияние атак на точность работы модели. Возьмем несколько epsilon для анализа.

### Epsilon = 0.05

In [ ]:
eps=0.05

# Getting data with fgsm attack
perturbed_data, origin_data = data_fgsm_attack(
    model, DEVICE, test_dataloader, eps
) 

evaluator = Evaluator(infer, perturbed_data)
predictions = evaluator.evaluate()
metrics =evaluator.classification_report()

print(f'Classification report with fgsm attack eps={eps}')
print(metrics)

Получим предсказания для оригинальныых изображений

In [ ]:
evaluator = Evaluator(infer, origin_data)
# Getting predictions 
predictions_without_attack = evaluator.evaluate()
# Getting metrics
source_metrics =evaluator.classification_report()

print(f'Classification report with origianl img')
print(source_metrics)

Посмотрим на оригинальные изображения и изображения с аттакой с epsilon = 0.05

In [ ]:
# Сохраним данные для визуализации при eps=0.05
eps_05=eps
predictions_05=predictions
perturbed_data_05=perturbed_data

In [ ]:
@interact
def show_predictions(index=IntSlider(val=0, min=0, max=len(perturbed_data)-1)):

    or_img,_= origin_data[index]
    or_img = np.array(or_img)
    pred_label_without_attack = predictions_without_attack.iloc[index,1]


    test_img_05,target_05= perturbed_data_05[index]
    test_img_05 = np.array(test_img_05)
    pred_label_05 = predictions_05.iloc[index,1]

    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[10, 10])

    ax[0].imshow(or_img,cmap='gray')
    ax[0].set_title("Original image")
    ax[1].imshow(test_img_05,cmap='gray')
    ax[1].set_title(f"Epsilon : {eps_05}")
    
    print(f"True Label: {target_05}")
    print(f"Predicted Label without attack: {pred_label_without_attack}")
    print(f"Predicted Label with fgsm attack: {pred_label_05}")
    plt.show()

### Epsilon = 0.1

In [ ]:
eps=0.1

# Getting data with fgsm attack
perturbed_data, origin_data = data_fgsm_attack(
    model, DEVICE, test_dataloader, eps
)
mnist_evaluator= Evaluator(infer, perturbed_data)
# Getting predictions 
predictions = mnist_evaluator.evaluate()
# Getting metrics
metrics =mnist_evaluator.classification_report()

print(f'Classification report with fgsm attack eps={eps}')
print(metrics)

In [ ]:
# Сохраним данные для визуализации 
eps_01 = eps
predictions_01 = predictions
perturbed_data_01 = perturbed_data

Посмотрим на изображение с epsilon = 0.1

In [ ]:
@interact
def show_predictions(index=IntSlider(val=0, min=0, max=len(perturbed_data)-1)):

    or_img,_= origin_data[index]
    or_img = np.array(or_img)
    pred_label_without_attack = predictions_without_attack.iloc[index,1]


    test_img_01,target_01= perturbed_data_01[index]
    test_img_01 = np.array(test_img_01)
    pred_label_01 = predictions_01.iloc[index,1]

    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[10, 10])

    ax[0].imshow(or_img,cmap='gray')
    ax[0].set_title("Original image")
    ax[1].imshow(test_img_01,cmap='gray')
    ax[1].set_title(f"Epsilon : {eps_01}")
    
    print(f"True Label: {target_01}")
    print(f"Predicted Label without attack: {pred_label_without_attack}")
    print(f"Predicted Label with attack: {pred_label_01}")
    plt.show()
